### **INTRODUCCIÓN A LA MINERÍA DE DATOS**
#### **Práctica 2: Clasificación**
- **Universidad de Córdoba**
- **Escuela Politécnica Superior de Córdoba**
- **Grado en Ingeniería Informática**
- **Mención: Computación**
- **Cuarto año - Primer cuatrimestre**
- **Autor:** Ventura Lucena Martínez
- **Email:** <i72lumav@uco.es>
---

#### **Data**
We are using the dataset [`weather.arff`](https://storm.cis.fordham.edu/~gweiss/data-mining/weka-data/weather.arff):

In [ ]:
from scipy.io import arff
import pandas as pd

# Load '.arff' file.
data = arff.loadarff("../datasets/vote.arff")
df = pd.DataFrame(data[0])

df
